# Overview

In this notebook, we'll be building a system that can read the data from a PDF or PNG file and then create a structured json payload, which we predefined its structure.

# Set up

In [ ]:
import os
from dotenv import load_dotenv; load_dotenv()

%load_ext autoreload
%autoreload 2

# Data model

We will be using Pydantic to define a data model, including fields and types, for the json payload extracted from a PDF/PNG file.

In [ ]:
from pydantic import BaseModel

In [ ]:
class Restaurant(BaseModel):
    """Data model for an restaurant."""
    restaurant: str
    food: str
    discount: str
    price: str
    rating: str
    review: str

# Load data

In [ ]:
from llama_index.core import SimpleDirectoryReader

In [ ]:
# Load images from the directory
image_documents = SimpleDirectoryReader("../data/restaurant_images").load_data()
image_documents

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# Plot a sample image
sample_image_doc = image_documents[1]
image = Image.open(sample_image_doc.metadata['file_path'])
plt.figure(figsize=(16, 5))
plt.imshow(image)
plt.axis('off')
plt.show()

# Extract data

In [ ]:
from llama_index.multi_modal_llms.openai import OpenAIMultiModal

In [ ]:
# Create a GPT-4V client using a wrapper from LlamaIndex
openai_mm_llm = OpenAIMultiModal(
    model="gpt-4-vision-preview", 
    api_key=os.getenv("OPENAI_API_KEY"), 
    max_new_tokens=1000,
)

In [ ]:
# Define a prompt string for data extraction
DATA_EXTRACT_STR = """\
Can you summarize what is in the image and return the answer with json format
"""

In [ ]:
from llama_index.core.program import MultiModalLLMCompletionProgram
from llama_index.core.output_parsers import PydanticOutputParser

In [ ]:
# Create a program to generate structured outputs using Multi-modal LLM + Output Parser
program = MultiModalLLMCompletionProgram.from_defaults(
    output_parser=PydanticOutputParser(Restaurant),
    image_documents=[sample_image_doc],
    prompt_template_str=DATA_EXTRACT_STR,
    multi_modal_llm=openai_mm_llm,
    verbose=True,
)

In [ ]:
import json

In [ ]:
# Run the program and print the result
response = program()
print(json.dumps(json.loads(response.json()), indent=2))